In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib ipympl
plt.rcParams['figure.figsize'] = [10, 4]
plt.rcParams['font.size'] = 8
mpl.rc('image', cmap='gray')
gs = gridspec.GridSpec(2, 2)

import matplotlib.animation
writervideo = matplotlib.animation.FFMpegWriter(fps=30)

import numpy as np
import pandas as pd
import csv, json

import pims
import trackpy as tp
tp.quiet()
from PIL import Image, ImageDraw
import cv2

from scipy.optimize import dual_annealing, minimize
from scipy.optimize import linear_sum_assignment

from tqdm import tqdm
import joblib

from scipy.spatial import distance_matrix
from scipy.ndimage import uniform_filter1d

import random

run_analysis_verb = False
show_verb = True
save_verb = True

In [11]:
@pims.pipeline
def hough_preprocessing(image, x1, y1, x2, y2):    
    #image = cv2.GaussianBlur(image, ksize = [7,7], sigmaX = 1.5, sigmaY = 1.5)
    npImage = np.array(image)
    # Create same size alpha layer with circle
    alpha = Image.new('L', (920, 960), 0)

    draw = ImageDraw.Draw(alpha)
    draw.pieslice(((x1, y1), (x2, y2)), 0, 360, fill=255)

    # Convert alpha Image to numpy array
    npAlpha = np.array(alpha)
    npImage = cv2.cvtColor(npImage, cv2.COLOR_BGR2GRAY)*npAlpha #npImage[:, :, 1] * npAlpha
    
    ind = np.where(npImage == 0)
    # npImage[200, 200] color of the border to swap with the black
    npImage[ind] = npImage[200, 200]
    npImage = cv2.medianBlur(npImage, 3)
    return npImage

In [78]:
# SETUP
preload_load_data = False # takes 20 min
merge_frame = 32269
data = hough_preprocessing(pims.open('./data/movie.mp4'), 40, 55, 895, 910)
if preload_load_data: 
    data_preload = list(data[:merge_frame])

startFrame = 0
endFrame = merge_frame
frames = np.arange(startFrame, endFrame, 1)
frames_opt = np.sort(random.sample(list(frames), 5000))
correct_n = 50
default_parameters = {"dp": 1.5, "minDist": 15, "param1": 100, "param2": 0.8, "minRadius": 15, "maxRadius": 25}

deprecated pixel format used, make sure you did set range correctly


In [76]:
def loc_frame(correct_n, frame, img, parameters):
	found_circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT_ALT, **parameters)
	if (found_circles is not None) and (found_circles.shape[1] == correct_n):
		return np.hstack((found_circles[0], (np.ones((correct_n, 1), dtype=int)*frame), np.ones((correct_n, 1), dtype=int)*correct_n))
	elif (found_circles is not None) and (found_circles.shape[1] != correct_n):
		return np.hstack((np.zeros((correct_n, 3)), (np.ones((correct_n, 1), dtype=int)*frame), np.ones((correct_n, 1), dtype=int)*found_circles.shape[1]))
	else:
		return np.hstack((np.zeros((correct_n, 3)), (np.ones((correct_n, 1), dtype=int)*frame), np.zeros((correct_n, 1), dtype=int)))

@joblib.delayed
def loc_frame_parallel(correct_n, frame, img, parameters):
	found_circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT_ALT, **parameters)
	if (found_circles is not None) and (found_circles.shape[1] == correct_n):
		return np.hstack((found_circles[0], (np.ones((correct_n, 1), dtype=int)*frame), np.ones((correct_n, 1), dtype=int)*correct_n))
	elif (found_circles is not None) and (found_circles.shape[1] != correct_n):
		return np.hstack((np.zeros((correct_n, 3)), (np.ones((correct_n, 1), dtype=int)*frame), np.ones((correct_n, 1), dtype=int)*found_circles.shape[1]))
	else:
		return np.hstack((np.zeros((correct_n, 3)), (np.ones((correct_n, 1), dtype=int)*frame), np.zeros((correct_n, 1), dtype=int)))

def hough_feature_location(data_preload, frames, correct_n, params, parallel_verb):
    if parallel_verb:
        parallel = joblib.Parallel(n_jobs = -1)
        temp = parallel(
            loc_frame_parallel(correct_n, frames[i], data_preload[i], params)
            for i in range(len(frames)) #tqdm(range(len(frames)) )
        )
        #print(np.array(temp).shape)
    else:
        temp = []
        for i in range(len(frames)):#tqdm(range(len(frames))):
            temp.append(loc_frame(correct_n, frames[i], data_preload[i], params))
        #print(np.array(temp).shape)
    temp = pd.DataFrame(np.array(temp).reshape(len(frames)*correct_n, 5), columns = ["x", "y", "d", "frame", "nDroplets"])
    err_frames = temp.loc[temp.nDroplets != correct_n].frame.unique().astype(int)
    loss = err_frames.shape[0]/frames.shape[0]
    return temp, err_frames, loss

def optimize_params(x, *args):
    data_preload, frames, correct_n = args
    params = {"dp":x[0], "minDist":int(x[1]), "param1":x[2], "param2":x[3], "minRadius":int(x[4]), "maxRadius":int(x[5])}
    _, _, loss = hough_feature_location(data_preload, frames, correct_n, params, False)
    # Save the current best score and set of parameters to a CSV file
    a = [loss, x[0], int(x[1]), x[2], x[3], int(x[4]), int(x[5])]
    with open('optimization_results.csv', mode = 'a', newline='') as file:
       writer = csv.writer(file)
       writer.writerow(a)
    print(a)
    return loss

def plot_optimization_results(opt_result_df, slot2):
    fig, ax = plt.subplots(1, 1)
    ax.plot(np.arange(0, len(opt_result_df.loss), 1), opt_result_df.loss, 'b-')
    ax.set_ylabel("loss", color = 'b') 
    ax1 = ax.twinx() 
    ax1.plot(np.arange(0, len(opt_result_df[slot2]), 1), opt_result_df[slot2], 'r.')
    ax1.set_ylabel(slot2, color='r')
    ax.grid()
    plt.show()
    return fig

In [79]:
# Clear the contents of the CSV file before starting the optimization
with open('optimization_results.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['loss', 'dp', 'minDist', 'param1', 'param1', 'minRadius', 'maxRadius'])

opt_result = dual_annealing(optimize_params, x0 = [2, 8, 90, 0.8, 10, 35], args = (data_preload, frames_opt, correct_n),\
                            bounds = [(1, 3), (5, 20), (80, 200), (0.3, 1), (5, 20), (20, 40)], \
                            maxiter=10, seed=1234)

opt_result

[0.0014, 2.0, 8, 90.0, 0.8, 10, 35]
[0.1592, 1.479039490222931, 13, 151.39343750476837, 0.7790394925718505, 11, 30]
[0.0406, 1.2710235863924026, 14, 133.78300543874502, 0.5710235813034725, 11, 32]
[0.9882, 2.5822854191064835, 12, 102.02556505613029, 0.3822854222973712, 10, 21]
[0.383, 2.7896605785936117, 7, 172.0742613542825, 0.38966057969888873, 15, 31]
[0.0108, 1.2978378981351852, 15, 88.58243867754936, 0.3978379145350556, 6, 27]
[0.0368, 1.1067238869145513, 6, 132.39132466632873, 0.3067239038544422, 16, 31]
[0.0336, 1.9494688361883163, 6, 132.39132466632873, 0.3067239038544422, 16, 31]
[0.0336, 1.9494688361883163, 6, 132.39132466632873, 0.3067239038544422, 16, 31]
[0.3388, 1.9494688361883163, 6, 167.50158191472292, 0.3067239038544422, 16, 31]
[0.3392, 1.9494688361883163, 6, 167.50158191472292, 0.4197892867302319, 16, 31]


In [ ]:
optimization_verb = True
run_optimization_verb = True
if optimization_verb:
    if run_optimization_verb:
        # Define a function to print the current best score and set of parameters
        def callback(x, f, context):
            print(f'Current score: {f}, Best parameters: {x}')
            # Save the current best score and set of parameters to a CSV file
            with open('optimization_results.csv', mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(list(x) + [f])

        # Clear the contents of the CSV file before starting the optimization
        with open('optimization_results.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['dp', 'minDist', 'param1', 'param2', 'minRadius', 'maxRadius', 'loss'])
        bounds = [(1, 3), (5, 20), (80, 200), (0.3, 1), (5, 20), (20, 40)]
        opt_result = dual_annealing(optimize_params, x0 = [1.5, 15, 200, 0.9, 15, 25], args = (data_preload, frames_opt[:100], correct_n),\
                                    bounds = bounds, maxiter = 1000, callback=callback)
    else:
        try:
            opt_result_df = pd.read_csv("./results/tracking_data/hough/pre_merge_optimization.csv", sep="\t").sort_values("loss", ascending=False)
            opt_result_df = opt_result_df.sort_values("loss", ascending=False)
            optimized_parameters = opt_result_df.iloc[-1]
            optimized_parameters = {"dp": optimized_parameters.dp, "minDist": optimized_parameters.minDist,\
                                    "param1": optimized_parameters.param1, "param2": optimized_parameters.param2,\
                                    "minRadius": int(optimized_parameters.minRadius), "maxRadius": int(optimized_parameters.maxRadius)}
            parameters = optimized_parameters
            print("Optimized parameters:", parameters)
            fig, ax = plt.subplots(1, 1, figsize = (10, 3))
            ax.plot(opt_result_df.loss.values)
            ax.set_ylabel("loss")
            ax.set_xlabel("iteration")
            ax.grid()
            plt.show()
        except:
            raise Exception("No optimization results found")
else:
    parameters = default_parameters
    print("Default parameters:", parameters)

In [ ]:
opt_params = 2.50993504   8.00993504 109.88808087   0.70993504  14.88808087 34.88808087

In [ ]:
def analyze_err_frame(trial_f):
    temp = cv2.HoughCircles(data_preload[trial_f], cv2.HOUGH_GRADIENT_ALT, **params)[0]

    fig, (ax, ax1) = plt.subplots(1, 2, figsize = (10, 5))
    ax.imshow(data_preload[trial_f])
    ax.set_title(f"Error frame: {trial_f} --> {temp.shape[0]} droplet found")
    for i in range(temp.shape[0]):
        ax.add_artist(plt.Circle((temp[i, 0], temp[i, 1]), temp[i, 2], color='r', fill=False))
    ax1.scatter(temp[:,2])
    plt.show()

In [ ]:
params = {"dp": .5, "minDist": 13, "param1": 80, "param2": 0.6, "minRadius": 11, "maxRadius": 29}
pre_merge_df, err_frames, error = hough_feature_location(data_preload, frames_opt, correct_n, params, False)
print("Error:", error)
print("Error frames:", err_frames)
if len(err_frames) > 0:
    analyze_err_frame(err_frames[10])
if len(err_frames) == 0:
    pre_merge_df, err_frames, error = hough_feature_location(data_preload, frames, correct_n, params, False)
    print("Error:", error)
    print("Error frames:", err_frames)

In [ ]:
if 1:
    # save to txt parameters:
    with open('./results/tracking_data/hough/hough_pre_merge.txt', 'w') as f:
        f.write(json.dumps(parameters))
    pre_merge_df, err_frames, error = hough_feature_location(data_preload, frames, correct_n, parameters, False)
    pre_merge_df.to_parquet("./results/tracking_data/hough_pre_merge.parquet")
else:
    try:
        parameters = json.load(open('./results/tracking_data/hough/hough_pre_merge.txt'))
        pre_merge_df = pd.read_parquet("./results/tracking_data/hough/hough_pre_merge.parquet")
        print(parameters)
        display(pre_merge_df)
    except:
        raise Exception("No pre merge data found, run analysis first")

In [ ]:
frames = (pre_merge_df.frame.values[::correct_n]).astype(int)
print(f"Number of frames: {len(frames)}")
err_frames = pre_merge_df.loc[pre_merge_df.nDroplets != correct_n].frame.unique().astype(int)
print(f"Percentage of error: {len(err_frames)}/{len(frames)}")